In [1]:
def PPLRC(i, j):

    import sys
    import numpy as np
    import concurrent.futures
    import queue
    import time

    np.set_printoptions(suppress=True)
    np.warnings.filterwarnings('ignore')
    np.set_printoptions(threshold=sys.maxsize)

    ###################################################################################

    def share(input):
        rs0 = np.random.RandomState(13)
        a = 0.0000001 * rs0.random((np.shape(input)))

        rs1 = np.random.RandomState(22)
        b = 0.0000001 * rs1.random((np.shape(input)))

        c = input - a - b

        return a, b, c

    def sharey(input):
        rs4 = np.random.RandomState(24)
        a = rs4.randint(0, 2, np.shape(input))

        rs5 = np.random.RandomState(45)
        b = rs5.randint(0, 2, np.shape(input))

        c = input - a - b

        return a, b, c

    ###################################################################################

    sizex = i
    sizey = j

    rs12 = np.random.RandomState(1)
    XA = np.round(rs12.uniform(-1, 1, (sizex, sizey)), 7)
#     XA1, XA2 = share(XA, 4)

    rs13 = np.random.RandomState(2)
    YA = rs13.randint(0, 2, (sizex, 1))
#     YA1, YA2 = sharey(YA)


    #     rs14 = np.random.RandomState(3)
    #     XB = np.round(rs14.uniform(-1, 1, (sizex, sizey)), 7)
    #     XB1, XB2 = share(XB, 6)

    #     rs15 = np.random.RandomState(4)
    #     YB = rs15.randint(0, 2, (sizex, 1))
    #     YB1, YB2 = sharey(YB)


    X_train = XA
    #     np.concatenate((XA, XB), axis=0)
    y_train = YA
    #     np.concatenate((YA, YB), axis=0)

    X_train1, X_train2, X_train3  = share(X_train)
    y_train1, y_train2, y_train3 = sharey(y_train)



    ###################################################################################


    def CP1(beta):
        def mul_multiply(first, second):

            r = rs5.uniform(0.0000000000001, 10 ** (-40), np.shape(first))
            j = rs5.uniform(0.0000000000001, 10 ** (-40), np.shape(second))

            q1.put(r)
            q1.put(j)

            k = q3.get()
            h = q3.get()

            A = first + r - k
            B = second + j - h

            q11.put(A)
            q11.put(B)

            C = q33.get()
            D = q33.get()

            W = np.multiply(A, B) + np.multiply(A, D) + np.multiply(C, B)
            return W

        def mul_multiply_number(first, second):

            # rs14 = np.random.RandomState(1)
            # r = rs14.uniform(0, 1, (np.shape(first)))
            # rs14 = np.random.RandomState(1)
            # j = rs14.uniform(0, 1, (np.shape(second)))
            np.random.seed(1)
            r = np.random.random((np.shape(first)))
            np.random.seed(1)
            j = np.random.uniform(0.01, 10 ** (-6), np.shape(second))

            q1.put(r)
            q1.put(j)

            k = q3.get()
            h = q3.get()

            A = first + r - k
            B = second + j - h

            q11.put(A)
            q11.put(B)

            C = q33.get()
            D = q33.get()

            W = (A * B) + (A * D) + (C * B)
            return W

        def mul_multiply_matrix_full(first, second):


            rs14 = np.random.RandomState(1)
            r = 0.0001 * rs14.random((np.shape(first)))
            rs14 = np.random.RandomState(1)
            j = 0.0001 *rs14.random((np.shape(second)))

            q1.put(r)
            q1.put(j)

            k = q3.get()
            h = q3.get()

            A = first + r - k
            B = second + j - h

            q11.put(A)
            q11.put(B)

            C = q33.get()
            D = q33.get()

            W = A.dot(B) + A.dot(D) + C.dot(B)

            return W

        X = X_train1
        y = y_train1
#         X = np.concatenate((XA1, XB1), axis=0)
#         y = np.concatenate((YA1, YB1), axis=0)

        # ///////////////////////////////////////Hessian///////////

        Hessian_temp = mul_multiply_matrix_full(X.T, X)

        # B = np.max(np.linalg.eigvals(Hessian_temp).real)
        #
        # c_0 = rs6.uniform(0.0000000000001, 10 ** (-40), (1, 1))
        # Xs = c_0 * 1 / 3
        # Ms = c_0 * B
        # # print(B)
        #
        #
        # for x in range(0, 20):
        #     Ms_Xs = mul_multiply(Xs, Ms)
        #     mul_2 = mul_multiply([[2 / 3]], Xs)
        #     X_s_t = mul_2 - Ms_Xs
        #
        #     Ms_Ms = mul_multiply(Ms, Ms)
        #     Ms_2 = mul_multiply([[2 / 3]], Ms)
        #     M_s_t = Ms_2 - Ms_Ms
        #
        #     Xs = X_s_t
        #     Ms = M_s_t

        Inve_eigen = [[0.00000002]]
        # print('\n',X_s_t)

        c = Inve_eigen
        q = np.size(Hessian_temp, 0)
        I = np.identity(q)
        Xs = mul_multiply(c, I / 3)
        Ms = mul_multiply(c, Hessian_temp)

        for x in range(0, 40):
            Ms_Xs = mul_multiply_matrix_full(Xs, Ms)
            mul_2 = mul_multiply([[2 / 3]], Xs)
            X_s_t = mul_2 - Ms_Xs

            Ms_Ms = mul_multiply_matrix_full(Ms, Ms)
            Ms_2 = mul_multiply([[2 / 3]], Ms)
            M_s_t = Ms_2 - Ms_Ms

            Xs = X_s_t
            Ms = M_s_t

        Hessian_Invers_TEMP = X_s_t

        Hessian_Invers = -4 * Hessian_Invers_TEMP

        # ///////////////////////////////////////Hessian///////////

        for i in range(0, 10):
            
            beta_old = beta

            XB = mul_multiply_matrix_full(X, beta_old) * 1 / 8

            ###################################################################################

            def Sig_5(XB):
            
                Ms1 = XB

                Ms2 = mul_multiply_number(XB, XB)
                Ms3 = mul_multiply_number(Ms2, XB)
                Ms4 = Ms3

                Ms5 = mul_multiply_number(XB, XB)
                Ms6 = mul_multiply_number(Ms5, XB)
                Ms7 = mul_multiply_number(Ms6, XB)
                Ms8 = mul_multiply_number(Ms7, XB)
                Ms9 = Ms8

                Sig = 1 / 2 + (1.53048 * Ms1) - (2.3533056 * Ms4) + (1.3511295 * Ms9)
                
                return Sig

            ####################################################################################

            def Sig_3(XB):
        
                Ms1 = XB

                Ms2 = mul_multiply_number(XB, XB)
                Ms3 = mul_multiply_number(Ms2, XB)
                Ms4 = Ms3

                Sig = 1/2 + (1.20096 * Ms1) - (0.81562 * Ms4)
                
                return Sig
            
            ###################################################################################

            def Sig_7(XB):
            
                Ms1 = XB

                Ms2 = mul_multiply_number(XB, XB)
                # print(XB)
                Ms3 = mul_multiply_number(Ms2, XB)
                Ms4 = Ms3

                Ms5 = mul_multiply_number(XB, XB)
                Ms6 = mul_multiply_number(Ms5, XB)
                Ms7 = mul_multiply_number(Ms6, XB)
                Ms8 = mul_multiply_number(Ms7, XB)
                Ms9 = Ms8

                Ms10 = mul_multiply_number(XB, XB)
                Ms11 = mul_multiply_number(Ms10, XB)
                Ms12 = mul_multiply_number(Ms11, XB)
                Ms13 = mul_multiply_number(Ms12, XB)
                Ms14 = mul_multiply_number(Ms13, XB)
                Ms15 = mul_multiply_number(Ms14, XB)

                Ms16 = Ms15

                Sig = 1/2 + (1.73496 * Ms1) - (4.19407 * Ms4) + (5.43402 * Ms9) - (2.50739 * Ms16)
                
                return Sig

            ###################################################################################

            grad_temp = y - Sig_7(XB)

            Gradient = mul_multiply_matrix_full(X.T, grad_temp)

            step = mul_multiply_matrix_full(Hessian_Invers, Gradient)

            beta = beta_old - step

        return beta


    ###################################################################################
    rs5 = np.random.RandomState(20)
    rs6 = np.random.RandomState(30)


    def CP2(beta):
        def mul_multiply(first, second):

            r = rs5.uniform(0.0000000000001, 10 ** (-40), np.shape(first))

            j = rs5.uniform(0.0000000000001, 10 ** (-40), np.shape(second))

            q2.put(r)
            q2.put(j)

            k = q1.get()
            h = q1.get()

            A = first + r - k
            B = second + j - h

            q22.put(A)
            q22.put(B)

            C = q11.get()
            D = q11.get()

            W = np.multiply(A, B) + np.multiply(A, D) + np.multiply(C, B)
            return W

        def mul_multiply_number(first, second):

            # rs14 = np.random.RandomState(1)
            # r = rs14.uniform(0, 1, (np.shape(first)))
            # rs14 = np.random.RandomState(1)
            # j = rs14.uniform(0, 1, (np.shape(second)))
            np.random.seed(2)  # set the seed
            r = np.random.random((np.shape(first)))

            np.random.seed(2)  # set the seed
            j = np.random.uniform(0.01, 10 ** (-6), np.shape(second))

            q2.put(r)
            q2.put(j)

            k = q1.get()
            h = q1.get()

            A = first + r - k
            B = second + j - h

            q22.put(A)
            q22.put(B)

            C = q11.get()
            D = q11.get()

            W = (A * B) + (A * D) + (C * B)
            return W

        def mul_multiply_matrix_full(first, second):
            # print(second)

            rs14 = np.random.RandomState(1)
            r = 0.0001 *rs14.random((np.shape(first)))
            rs14 = np.random.RandomState(1)
            j = 0.0001 *rs14.random((np.shape(second)))

            q2.put(r)
            q2.put(j)

            k = q1.get()
            h = q1.get()

            A = first + r - k
            B = second + j - h

            q22.put(A)
            q22.put(B)

            C = q11.get()
            D = q11.get()

            W = A.dot(B) + A.dot(D) + C.dot(B)
            return W

        X = X_train2
        y = y_train2

#         X = np.concatenate((XA2, XB2), axis=0)
#         y = np.concatenate((YA2, YB2), axis=0)

        # ///////////////////////////////////////Hessian///////////

        Hessian_temp = mul_multiply_matrix_full(X.T, X)

        # B = np.max(np.linalg.eigvals(Hessian_temp).real)
        # # print(B)
        #
        # c_0 = rs6.uniform(0.0000000000001, 10 ** (-40), (1, 1))
        # Xs = c_0 * 1 / 3
        # Ms = c_0 * B
        #
        # for x in range(0, 20):
        #     Ms_Xs = mul_multiply(Xs, Ms)
        #     mul_2 = mul_multiply([[2 / 3]], Xs)
        #     X_s_t = mul_2 - Ms_Xs
        #
        #     Ms_Ms = mul_multiply(Ms, Ms)
        #     Ms_2 = mul_multiply([[2 / 3]], Ms)
        #     M_s_t = Ms_2 - Ms_Ms
        #
        #     Xs = X_s_t
        #     Ms = M_s_t

        Inve_eigen = [[0.00000002]]
        # print('\n',X_s_t)

        c = Inve_eigen
        q = np.size(Hessian_temp, 0)
        I = np.identity(q)
        Xs = mul_multiply(c, I / 3)
        Ms = mul_multiply(c, Hessian_temp)
        # print(Hessian_temp)
        # print(Ms)

        for x in range(0, 40):
            Ms_Xs = mul_multiply_matrix_full(Xs, Ms)
            mul_2 = mul_multiply([[2 / 3]], Xs)
            X_s_t = mul_2 - Ms_Xs

            Ms_Ms = mul_multiply_matrix_full(Ms, Ms)
            Ms_2 = mul_multiply([[2 / 3]], Ms)
            M_s_t = Ms_2 - Ms_Ms
            # print(M_s_t)
            Xs = X_s_t
            Ms = M_s_t

        Hessian_Invers_TEMP = X_s_t

        Hessian_Invers = -4 * Hessian_Invers_TEMP

        # ///////////////////////////////////////Hessian///////////

        for i in range(0, 10):
            beta_old = beta

            XB = mul_multiply_matrix_full(X, beta_old) * 1 / 8

            ###################################################################################

            def Sig_5(XB):
            
                Ms1 = XB

                Ms2 = mul_multiply_number(XB, XB)
                Ms3 = mul_multiply_number(Ms2, XB)
                Ms4 = Ms3

                Ms5 = mul_multiply_number(XB, XB)
                Ms6 = mul_multiply_number(Ms5, XB)
                Ms7 = mul_multiply_number(Ms6, XB)
                Ms8 = mul_multiply_number(Ms7, XB)
                Ms9 = Ms8

                Sig = (1.53048 * Ms1) - (2.3533056 * Ms4) + (1.3511295 * Ms9)
                
                return Sig

            # ###################################################################################

            def Sig_3(XB):
        
                Ms1 = XB

                Ms2 = mul_multiply_number(XB, XB)
                Ms3 = mul_multiply_number(Ms2, XB)
                Ms4 = Ms3

                Sig = (1.20096 * Ms1) - (0.81562 * Ms4)
                
                return Sig
            
            ###################################################################################

            def Sig_7(XB):
            
                Ms1 = XB

                Ms2 = mul_multiply_number(XB, XB)
                # print(XB)
                Ms3 = mul_multiply_number(Ms2, XB)
                Ms4 = Ms3

                Ms5 = mul_multiply_number(XB, XB)
                Ms6 = mul_multiply_number(Ms5, XB)
                Ms7 = mul_multiply_number(Ms6, XB)
                Ms8 = mul_multiply_number(Ms7, XB)
                Ms9 = Ms8

                Ms10 = mul_multiply_number(XB, XB)
                Ms11 = mul_multiply_number(Ms10, XB)
                Ms12 = mul_multiply_number(Ms11, XB)
                Ms13 = mul_multiply_number(Ms12, XB)
                Ms14 = mul_multiply_number(Ms13, XB)
                Ms15 = mul_multiply_number(Ms14, XB)

                Ms16 = Ms15

                Sig = (1.73496 * Ms1) - (4.19407 * Ms4) + (5.43402 * Ms9) - (2.50739 * Ms16)
                
                return Sig

            ###################################################################################

            grad_temp = y - Sig_7(XB)

            Gradient = mul_multiply_matrix_full(X.T, grad_temp)

            step = mul_multiply_matrix_full(Hessian_Invers, Gradient)

            beta = beta_old - step

        return beta


    ###################################################################################
    rs5 = np.random.RandomState(20)
    rs6 = np.random.RandomState(30)


    def CP3(beta):
        def mul_multiply(first, second):

            r = rs5.uniform(0.0000000000001, 10 ** (-40), np.shape(first))

            j = rs5.uniform(0.0000000000001, 10 ** (-40), np.shape(second))

            q3.put(r)
            q3.put(j)

            k = q2.get()
            h = q2.get()

            A = first + r - k
            B = second + j - h

            q33.put(A)
            q33.put(B)

            C = q22.get()
            D = q22.get()

            W = np.multiply(A, B) + np.multiply(A, D) + np.multiply(C, B)
            return W

        def mul_multiply_number(first, second):

            np.random.seed(3)
            r = np.random.random((np.shape(first)))

            np.random.seed(3)
            j = np.random.uniform(0.01, 10 ** (-6), np.shape(second))

            q3.put(r)
            q3.put(j)

            k = q2.get()
            h = q2.get()

            A = first + r - k
            B = second + j - h

            q33.put(A)
            q33.put(B)

            C = q22.get()
            D = q22.get()

            W = (A * B) + (A * D) + (C * B)
            return W

        def mul_multiply_matrix_full(first, second):

            rs14 = np.random.RandomState(1)
            r = 0.0001 *rs14.random((np.shape(first)))
            rs14 = np.random.RandomState(1)
            j = 0.0001 *rs14.random((np.shape(second)))

            q3.put(r)
            q3.put(j)

            k = q2.get()
            h = q2.get()

            A = first + r - k
            B = second + j - h

            q33.put(A)
            q33.put(B)

            C = q22.get()
            D = q22.get()

            W = A.dot(B) + A.dot(D) + C.dot(B)
            return W

        X = X_train3
        y = y_train3

#         X = np.concatenate((XA3, XB3), axis=0)
#         y = np.concatenate((YA3, YB3), axis=0)

        # ///////////////////////////////////////Hessian///////////

        Hessian_temp = mul_multiply_matrix_full(X.T, X)
        # print(Hessian_temp)

        # B = np.max(np.linalg.eigvals(Hessian_temp).real)
        # # print(B)
        #
        # c_0 = rs5.uniform(0.0000000000001, 10 ** (-40), (1, 1))
        # Xs = c_0 * 1 / 3
        # Ms = c_0 * B
        #
        # for x in range(0, 20):
        #     Ms_Xs = mul_multiply(Xs, Ms)
        #     mul_2 = mul_multiply([[2 / 3]], Xs)
        #     X_s_t = mul_2 - Ms_Xs
        #
        #     Ms_Ms = mul_multiply(Ms, Ms)
        #     Ms_2 = mul_multiply([[2 / 3]], Ms)
        #     M_s_t = Ms_2 - Ms_Ms
        #
        #     Xs = X_s_t
        #     Ms = M_s_t

        # print('\n',X_s_t)

        Inve_eigen = [[0.00000002]]

        c = Inve_eigen
        q = np.size(Hessian_temp, 0)
        I = np.identity(q)
        Xs = mul_multiply(c, I / 3)
        Ms = mul_multiply(c, Hessian_temp)

        for x in range(0, 40):
            Ms_Xs = mul_multiply_matrix_full(Xs, Ms)
            # print('Ms_Xs \n', Ms_Xs)
            mul_2 = mul_multiply([[2 / 3]], Xs)
            # print('mul_2 \n', mul_2)
            X_s_t = mul_2 - Ms_Xs
            # print('X_s_t \n', X_s_t)
            # print('############')

            Ms_Ms = mul_multiply_matrix_full(Ms, Ms)
            # print('Ms_Ms \n', Ms_Ms)
            Ms_2 = mul_multiply([[2 / 3]], Ms)
            # print('Ms_2 \n', Ms_2)
            M_s_t = Ms_2 - Ms_Ms
            # print('M_s_t \n', M_s_t)
            # print('#################')
            Xs = X_s_t
            Ms = M_s_t

        Hessian_Invers_TEMP = X_s_t

        Hessian_Invers = -4 * Hessian_Invers_TEMP

        # ///////////////////////////////////////Hessian///////////

        for i in range(0, 10):
            beta_old = beta

            XB = mul_multiply_matrix_full(X, beta_old) * 1 / 8

            ##################################################################################

            def Sig_5(XB):
            
                Ms1 = XB

                Ms2 = mul_multiply_number(XB, XB)
                Ms3 = mul_multiply_number(Ms2, XB)
                Ms4 = Ms3

                Ms5 = mul_multiply_number(XB, XB)
                Ms6 = mul_multiply_number(Ms5, XB)
                Ms7 = mul_multiply_number(Ms6, XB)
                Ms8 = mul_multiply_number(Ms7, XB)
                Ms9 = Ms8

                Sig = (1.53048 * Ms1) - (2.3533056 * Ms4) + (1.3511295 * Ms9)
                
                return Sig

            # ###################################################################################

            def Sig_3(XB):
        
                Ms1 = XB

                Ms2 = mul_multiply_number(XB, XB)
                Ms3 = mul_multiply_number(Ms2, XB)
                Ms4 = Ms3

                Sig = (1.20096 * Ms1) - (0.81562 * Ms4)
                
                return Sig

            ###################################################################################
              
            def Sig_7(XB):
                
                Ms1 = XB

                Ms2 = mul_multiply_number(XB, XB)
                Ms3 = mul_multiply_number(Ms2, XB)
                Ms4 = Ms3
                # print(XB)
                Ms5 = mul_multiply_number(XB, XB)
                Ms6 = mul_multiply_number(Ms5, XB)
                Ms7 = mul_multiply_number(Ms6, XB)
                Ms8 = mul_multiply_number(Ms7, XB)
                Ms9 = Ms8

                Ms10 = mul_multiply_number(XB, XB)
                Ms11 = mul_multiply_number(Ms10, XB)
                Ms12 = mul_multiply_number(Ms11, XB)
                Ms13 = mul_multiply_number(Ms12, XB)
                Ms14 = mul_multiply_number(Ms13, XB)
                Ms15 = mul_multiply_number(Ms14, XB)

                Ms16 = Ms15

                Sig = (1.73496 * Ms1) - (4.19407 * Ms4) + (5.43402 * Ms9) - (2.50739 * Ms16)
            
                return Sig

            ###################################################################################

            grad_temp = y - Sig_7(XB)

            Gradient = mul_multiply_matrix_full(X.T, grad_temp)

            step = mul_multiply_matrix_full(Hessian_Invers, Gradient)

            beta = beta_old - step

        return beta

    ###################################################################################

    q1 = queue.Queue()
    q2 = queue.Queue()
    q3 = queue.Queue()

    q11 = queue.Queue()
    q22 = queue.Queue()
    q33 = queue.Queue()

    q12 = queue.Queue()
    q13 = queue.Queue()

    q21 = queue.Queue()
    q23 = queue.Queue()

    q31 = queue.Queue()
    q32 = queue.Queue()

    ###################################################################################

    import atexit
    from time import time, strftime, localtime
    from datetime import timedelta

    def SMC():
#         beta = np.zeros((np.size(XA + XB, 1), 1))
        beta_old, beta = np.zeros((np.size(X_train, 1), 1)), np.zeros((np.size(X_train, 1), 1))
        beta1, beta2, beta3 = share(beta)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            f1 = executor.submit(CP1, beta1)
            f2 = executor.submit(CP2, beta2)
            f3 = executor.submit(CP3, beta3)

            Total = (f1.result() + f2.result() + f3.result())

            # print('Beta1' , '\n', f1.result(), '\n')

            # print('Beta2' , '\n', f2.result(), '\n')
            #
            # print('Beta3' , '\n', f3.result(), '\n')
            #
            # print('Total' , '\n', Total, '\n')

            return Total

        return ret



    start = time()
    beta = SMC()
    # print(beta)
    end = time()
    elapsed = end - start
#     print('Elapsed :', end - start)
    return beta

def mem1():

    beta1 = PPLRC(50000,200)  
    
#     print(beta1)
    
    return beta1

global m

m = 0

from memory_profiler import memory_usage

for x in range(0, 10): 

    mem = max(memory_usage(proc=mem1))
    
    m  = m + mem

    print("Maximum memory used: {} MiB".format(mem))
    
print(m/10)



Maximum memory used: 5374.328125 MiB
Maximum memory used: 5414.515625 MiB
Maximum memory used: 5424.78125 MiB
Maximum memory used: 5435.234375 MiB
Maximum memory used: 5438.734375 MiB
Maximum memory used: 5445.1875 MiB
Maximum memory used: 5450.40625 MiB
Maximum memory used: 5455.875 MiB
Maximum memory used: 5466.75 MiB
Maximum memory used: 5472.40625 MiB
5437.821875
